In [1]:
# =========================
# --- CELL 1: SETUP ---
# =========================
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm
import warnings

# Cài đặt thư viện tokenizers nếu chưa có
try:
    from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
except ImportError:
    !pip -q install tokenizers
    from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers

warnings.filterwarnings("ignore")

# --- Fix seed ---
def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device:", device)

# --- PATHS (ANH SỬA 2 DÒNG NÀY) ---
# 1) checkpoint best_transformer_en_vi.pt nằm trong Kaggle Input dataset nào thì trỏ vào đó
CKPT_PATH = "/kaggle/input/modeltrained/best_transformer_en_vi (3).pt"

# 2) data mới train.en/train.vi nằm trong Kaggle Input dataset nào thì trỏ vào đó
DATA_DIR = "/kaggle/input/databaitoanphu"

SRC_PATH = os.path.join(DATA_DIR, "train.en.txt")
TGT_PATH = os.path.join(DATA_DIR, "train.vi.txt")

print("CKPT_PATH:", CKPT_PATH)
print("SRC_PATH:", SRC_PATH)
print("TGT_PATH:", TGT_PATH)


✅ Device: cuda
CKPT_PATH: /kaggle/input/modeltrained/best_transformer_en_vi (3).pt
SRC_PATH: /kaggle/input/databaitoanphu/train.en.txt
TGT_PATH: /kaggle/input/databaitoanphu/train.vi.txt


In [2]:
# ==================================
# --- CELL 2: DATA READING FUNCTION ---
# ==================================
def read_parallel_data(src_path, tgt_path, max_lines=None):
    pairs = []
    with open(src_path, "r", encoding="utf-8") as fsrc, open(tgt_path, "r", encoding="utf-8") as ftgt:
        for i, (s, t) in enumerate(zip(fsrc, ftgt)):
            if max_lines and i >= max_lines:
                break
            s, t = s.strip(), t.strip()
            if s and t:
                pairs.append((s, t))
    return pairs


In [3]:
# =========================
# --- CELL 3: LOAD DATA ---
# =========================
print("\n--- LOAD DATA MỚI (train.en / train.vi) ---")

all_pairs = read_parallel_data(SRC_PATH, TGT_PATH, max_lines=None)
print("Tổng số cặp câu:", len(all_pairs))

# Shuffle + split 90% train, 10% val
random.shuffle(all_pairs)
split_idx = int(0.9 * len(all_pairs))
train_pairs = all_pairs[:split_idx]
val_pairs   = all_pairs[split_idx:]

print("Train pairs:", len(train_pairs))
print("Val pairs  :", len(val_pairs))



--- LOAD DATA MỚI (train.en / train.vi) ---
Tổng số cặp câu: 500000
Train pairs: 450000
Val pairs  : 50000


In [4]:
# ==============================
# --- CELL 4: TRAIN TOKENIZERS ---
# ==============================
print("\n--- HUẤN LUYỆN TOKENIZER ---")

def train_bpe_tokenizer(texts, vocab_size=8000):
    tokenizer = Tokenizer(models.BPE())
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
    tokenizer.decoder = decoders.ByteLevel()
    
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_size,
        special_tokens=["[PAD]", "[START]", "[END]", "[UNK]"],
        show_progress=False
    )
    tokenizer.train_from_iterator(texts, trainer=trainer)
    return tokenizer

all_src_text = [p[0] for p in train_pairs + val_pairs]
all_tgt_text = [p[1] for p in train_pairs + val_pairs]

if not all_src_text:  # Dummy data nếu chưa load được file
    all_src_text = ["Hello world"]
    all_tgt_text = ["Xin chào"]

en_tokenizer = train_bpe_tokenizer(all_src_text, vocab_size=10000)
vi_tokenizer = train_bpe_tokenizer(all_tgt_text, vocab_size=10000)

# Lấy ID các token đặc biệt
PAD_ID = en_tokenizer.token_to_id("[PAD]")
START_ID = vi_tokenizer.token_to_id("[START]")
END_ID = vi_tokenizer.token_to_id("[END]")

print("✅ Tokenizer đã sẵn sàng.")
print("PAD_ID:", PAD_ID, "START_ID:", START_ID, "END_ID:", END_ID)
print("SRC vocab:", en_tokenizer.get_vocab_size(), "TGT vocab:", vi_tokenizer.get_vocab_size())



--- HUẤN LUYỆN TOKENIZER ---
✅ Tokenizer đã sẵn sàng.
PAD_ID: 0 START_ID: 1 END_ID: 2
SRC vocab: 10000 TGT vocab: 10000


In [5]:
# ===========================
# --- CELL 5: DATASET CLASS ---
# ===========================
class TranslationDataset(Dataset):
    def __init__(self, pairs, src_tokenizer, tgt_tokenizer, max_len=64):
        self.pairs = pairs
        self.src_tok = src_tokenizer
        self.tgt_tok = tgt_tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        src, tgt = self.pairs[idx]

        src_ids = self.src_tok.encode(src).ids[:self.max_len]
        tgt_ids = self.tgt_tok.encode(tgt).ids[:self.max_len]

        # Thêm START/END cho tgt
        tgt_ids = [START_ID] + tgt_ids + [END_ID]

        return torch.tensor(src_ids, dtype=torch.long), torch.tensor(tgt_ids, dtype=torch.long)

def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_batch = nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=PAD_ID)
    tgt_batch = nn.utils.rnn.pad_sequence(tgt_batch, batch_first=True, padding_value=PAD_ID)
    return src_batch, tgt_batch


In [6]:
# ============================
# --- CELL 6: DATALOADERS ---
# ============================
BATCH_SIZE = 64

train_ds = TranslationDataset(train_pairs, en_tokenizer, vi_tokenizer, max_len=64)
val_ds   = TranslationDataset(val_pairs, en_tokenizer, vi_tokenizer, max_len=64)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, drop_last=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print("✅ DataLoader ready.")
print("Train batches:", len(train_loader), "Val batches:", len(val_loader))


✅ DataLoader ready.
Train batches: 7031 Val batches: 782


In [7]:
# ===================================
# --- CELL 7: MODEL COMPONENTS ---
# ===================================
# --- Rotary Positional Embeddings ---
def rotate_half(x):
    x1, x2 = x.chunk(2, dim=-1)
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(x, cos, sin):
    return (x * cos) + (rotate_half(x) * sin)

class RotaryPositionalEncoding(nn.Module):
    def __init__(self, head_dim, max_seq_len=2048):
        super().__init__()
        inv_freq = 1.0 / (10000 ** (torch.arange(0, head_dim, 2).float() / head_dim))
        t = torch.arange(max_seq_len).float()
        freqs = torch.outer(t, inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos", emb.cos()[None, None, :, :])
        self.register_buffer("sin", emb.sin()[None, None, :, :])

    def forward(self, x, seq_len):
        return self.cos[:, :, :seq_len, :], self.sin[:, :, :seq_len, :]

# --- SwiGLU ---
class SwiGLU(nn.Module):
    def __init__(self, hidden_dim, ffn_dim):
        super().__init__()
        self.w1 = nn.Linear(hidden_dim, ffn_dim)
        self.w2 = nn.Linear(hidden_dim, ffn_dim)
        self.w3 = nn.Linear(ffn_dim, hidden_dim)

    def forward(self, x):
        return self.w3(F.silu(self.w1(x)) * self.w2(x))

# --- GQA Attention ---
class GQA(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_kv_heads, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.num_kv_heads = num_kv_heads
        self.head_dim = hidden_dim // num_heads
        self.num_groups = num_heads // num_kv_heads
        
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, num_kv_heads * self.head_dim)
        self.v_proj = nn.Linear(hidden_dim, num_kv_heads * self.head_dim)
        self.o_proj = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = dropout

    def forward(self, x, enc_out=None, mask=None, rope_cos=None, rope_sin=None):
        batch, seq_len, _ = x.shape
        kv_input = enc_out if enc_out is not None else x

        q = self.q_proj(x).view(batch, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(kv_input).view(batch, -1, self.num_kv_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(kv_input).view(batch, -1, self.num_kv_heads, self.head_dim).transpose(1, 2)

        if rope_cos is not None:
            q = apply_rotary_pos_emb(q, rope_cos, rope_sin)
            k = apply_rotary_pos_emb(k, rope_cos, rope_sin)

        k = k.repeat_interleave(self.num_groups, dim=1)
        v = v.repeat_interleave(self.num_groups, dim=1)

        attn_scores = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)

        if mask is not None:
            attn_scores = attn_scores + mask

        attn = F.softmax(attn_scores, dim=-1)
        attn = F.dropout(attn, p=self.dropout, training=self.training)

        out = attn @ v
        out = out.transpose(1, 2).contiguous().view(batch, seq_len, -1)
        return self.o_proj(out)


In [8]:
# ==================================
# --- CELL 8: TRANSFORMER MODEL ---
# ==================================
class TransformerBlock(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_kv_heads, dropout=0.1, is_decoder=False):
        super().__init__()
        self.norm1 = nn.RMSNorm(hidden_dim)
        self.attn = GQA(hidden_dim, num_heads, num_kv_heads, dropout)
        self.is_decoder = is_decoder
        if is_decoder:
            self.norm2 = nn.RMSNorm(hidden_dim)
            self.cross_attn = GQA(hidden_dim, num_heads, num_kv_heads, dropout)
        self.norm_ffn = nn.RMSNorm(hidden_dim)
        self.ffn = SwiGLU(hidden_dim, hidden_dim * 4)

    def forward(self, x, enc_out=None, mask=None, cross_mask=None, rope_cos=None, rope_sin=None):
        x = x + self.attn(self.norm1(x), mask=mask, rope_cos=rope_cos, rope_sin=rope_sin)
        if self.is_decoder:
            x = x + self.cross_attn(self.norm2(x), enc_out=enc_out, mask=cross_mask)
        x = x + self.ffn(self.norm_ffn(x))
        return x

class Transformer(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, hidden_dim=256, num_layers=4, num_heads=8, num_kv_heads=4):
        super().__init__()
        self.src_emb = nn.Embedding(src_vocab, hidden_dim)
        self.tgt_emb = nn.Embedding(tgt_vocab, hidden_dim)
        self.rope = RotaryPositionalEncoding(hidden_dim // num_heads)
        
        self.encoders = nn.ModuleList([
            TransformerBlock(hidden_dim, num_heads, num_kv_heads, is_decoder=False)
            for _ in range(num_layers)
        ])
        self.decoders = nn.ModuleList([
            TransformerBlock(hidden_dim, num_heads, num_kv_heads, is_decoder=True)
            for _ in range(num_layers)
        ])
        self.final_norm = nn.RMSNorm(hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, tgt_vocab)

    def forward(self, src, tgt, src_mask, tgt_mask):
        x = self.src_emb(src)
        rope_cos, rope_sin = self.rope(x, x.shape[1])
        for layer in self.encoders:
            x = layer(x, mask=src_mask, rope_cos=rope_cos, rope_sin=rope_sin)
        enc_out = x
        
        x = self.tgt_emb(tgt)
        rope_cos_tgt, rope_sin_tgt = self.rope(x, x.shape[1])
        for layer in self.decoders:
            x = layer(x, enc_out=enc_out, mask=tgt_mask, cross_mask=src_mask, rope_cos=rope_cos_tgt, rope_sin=rope_sin_tgt)
        return self.fc_out(self.final_norm(x))


In [9]:
# ============================
# --- CELL 9: INIT TRAINING ---
# ============================
def create_masks(src, tgt):
    src_mask = (src == PAD_ID).unsqueeze(1).unsqueeze(2).float() * -1e9
    batch, seq_len = tgt.shape
    causal = torch.triu(torch.full((seq_len, seq_len), float('-inf'), device=device), diagonal=1)
    tgt_pad = (tgt == PAD_ID).unsqueeze(1).unsqueeze(2).float() * -1e9
    return src_mask, causal + tgt_pad

model = Transformer(
    src_vocab=en_tokenizer.get_vocab_size(),
    tgt_vocab=vi_tokenizer.get_vocab_size(),
    hidden_dim=256, num_layers=4, num_heads=8, num_kv_heads=4
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID, label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.0001)

print("Model Initialized.")


Model Initialized.


In [10]:
# ==========================================
# --- CELL 10: LOAD BEST CHECKPOINT + RESUME ---
# ==========================================
print("\n--- LOAD best_transformer_en_vi.pt & TRAIN TIẾP ---")

ckpt = torch.load(CKPT_PATH, map_location=device)

# ckpt đúng format từ notebook cũ:
# {"epoch":..., "model_state_dict":..., "optimizer_state_dict":..., "val_loss":...}

state_dict = ckpt["model_state_dict"]

# BỎ 2 buffer RoPE bị lệch shape giữa checkpoint và model hiện tại
for k in ["rope.cos", "rope.sin"]:
    if k in state_dict:
        state_dict.pop(k)

# load với strict=False để chấp nhận missing đúng 2 key trên
model.load_state_dict(state_dict, strict=False)

# resume optimizer nếu anh muốn train tiếp đúng trạng thái (khuyến nghị)
if "optimizer_state_dict" in ckpt:
    optimizer.load_state_dict(ckpt["optimizer_state_dict"])
    # đảm bảo optimizer states nằm đúng device
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                state[k] = v.to(device)

start_epoch = ckpt.get("epoch", 0)
best_prev_val = ckpt.get("val_loss", None)

print(f"✅ Loaded checkpoint from: {CKPT_PATH}")
print(f"   start_epoch = {start_epoch}")
print(f"   prev_best_val_loss = {best_prev_val}")



--- LOAD best_transformer_en_vi.pt & TRAIN TIẾP ---
✅ Loaded checkpoint from: /kaggle/input/modeltrained/best_transformer_en_vi (3).pt
   start_epoch = 7
   prev_best_val_loss = 3.338132079766721


In [11]:
# =========================
# --- CELL 11: EARLY STOP ---
# =========================
import math
import torch

class EarlyStopping:
    def __init__(self, patience=3, min_delta=1e-4, mode="min"):
        """
        mode="min": metric càng nhỏ càng tốt (val_loss)
        """
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode

        self.best = None
        self.num_bad_epochs = 0

    def _is_improvement(self, current):
        if self.best is None:
            return True
        if self.mode == "min":
            return current < (self.best - self.min_delta)
        else:
            return current > (self.best + self.min_delta)

    def step(self, current):
        if self._is_improvement(current):
            self.best = current
            return False
        else:
            self.num_bad_epochs += 1
            print(f"Số epoch kém chất lượng {self.num_bad_epochs}" )
            return self.num_bad_epochs >= self.patience


def save_checkpoint(path, model, optimizer, epoch, val_loss):
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "val_loss": val_loss,
    }, path)


In [15]:
# ======================================
# --- CELL 12: CONTINUE TRAINING LOOP ---
# ======================================
import time 
EPOCHS_MORE = 30                 # train tiếp thêm bao nhiêu epoch tuỳ anh
early = EarlyStopping(patience=3, min_delta=1e-4, mode="min")

# nếu checkpoint có best val trước đó thì set làm mốc
if best_prev_val is not None:
    early.best = best_prev_val

best_path = "best_transformer_en_vi_resume.pt"

print("\n--- BẮT ĐẦU HUẤN LUYỆN TIẾP ---")

for ep in range(EPOCHS_MORE):
    epoch = start_epoch + ep  # epoch thực tính tiếp
    t0 = time.time() 
    model.train()
    train_loss = 0
    pbar = tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/{start_epoch+EPOCHS_MORE}")

    for src, tgt in pbar:
        src, tgt = src.to(device), tgt.to(device)
        tgt_input, tgt_real = tgt[:, :-1], tgt[:, 1:]
        src_mask, tgt_mask = create_masks(src, tgt_input)

        optimizer.zero_grad()
        output = model(src, tgt_input, src_mask, tgt_mask)

        loss = criterion(output.reshape(-1, output.shape[-1]), tgt_real.reshape(-1))
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        train_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for src, tgt in val_loader:
            src, tgt = src.to(device), tgt.to(device)
            tgt_input, tgt_real = tgt[:, :-1], tgt[:, 1:]
            src_mask, tgt_mask = create_masks(src, tgt_input)
            output = model(src, tgt_input, src_mask, tgt_mask)
            val_loss += criterion(output.reshape(-1, output.shape[-1]), tgt_real.reshape(-1)).item()

    avg_val = val_loss / len(val_loader)
    epoch_time = time.time() - t0
    m, s = divmod(epoch_time, 60)
    h, m = divmod(m, 60)
    print(f"Epoch {epoch+1} | Train Loss: {avg_train:.4f} | Val Loss: {avg_val:.4f} | Time: {int(h):02d}:{int(m):02d}:{s:05.2f}")

    # save best + early stopping
    if early.best is None or avg_val < early.best - early.min_delta:
        save_checkpoint(best_path, model, optimizer, epoch+1, avg_val)
        print(f"  ↳ Saved BEST to {best_path} (val_loss={avg_val:.4f})")

    if early.step(avg_val):
        print(f"🛑 Early stopping at epoch {epoch+1}. Best val_loss = {early.best:.4f}")
        break

# (tuỳ chọn) vẫn lưu last state
torch.save(model.state_dict(), "last_transformer_en_vi_resume.pth")
print("✅ Done. Saved: best_transformer_en_vi_resume.pt & last_transformer_en_vi_resume.pth")



--- BẮT ĐẦU HUẤN LUYỆN TIẾP ---


Epoch 8/37: 100%|██████████| 7031/7031 [12:52<00:00,  9.11it/s, loss=2.7842]


Epoch 8 | Train Loss: 3.2456 | Val Loss: 2.7712 | Time: 00:13:24.62
  ↳ Saved BEST to best_transformer_en_vi_resume.pt (val_loss=2.7712)


Epoch 9/37: 100%|██████████| 7031/7031 [12:45<00:00,  9.18it/s, loss=2.5473]


Epoch 9 | Train Loss: 2.6633 | Val Loss: 2.5809 | Time: 00:13:17.36
  ↳ Saved BEST to best_transformer_en_vi_resume.pt (val_loss=2.5809)


...

Epoch 35 | Train Loss: 1.9260 | Val Loss: 2.2210 | Time: 00:13:13.00
  ↳ Saved BEST to best_transformer_en_vi_resume.pt (val_loss=2.2210)


Epoch 36/37: 100%|██████████| 7031/7031 [12:39<00:00,  9.25it/s, loss=1.9553]


Epoch 36 | Train Loss: 1.9186 | Val Loss: 2.2194 | Time: 00:13:11.42
  ↳ Saved BEST to best_transformer_en_vi_resume.pt (val_loss=2.2194)


Epoch 37/37: 100%|██████████| 7031/7031 [12:38<00:00,  9.26it/s, loss=1.9509]


Epoch 37 | Train Loss: 1.9117 | Val Loss: 2.2168 | Time: 00:13:10.74
  ↳ Saved BEST to best_transformer_en_vi_resume.pt (val_loss=2.2168)
✅ Done. Saved: best_transformer_en_vi_resume.pt & last_transformer_en_vi_resume.pth


In [43]:
# ==========================================
# LOAD BEST CHECKPOINT
# ==========================================
import os, torch

# CKPT_PATH = "best_transformer_en_vi_resume.pt"  (đây là trạng thái khi train hết 37 epoch)
CKPT_PATH = "/kaggle/input/startstatus/best_transformer_en_vi_resume.pt" # đây là trạng thái ở epoch: 23 val_loss: 2.261958536589542
assert os.path.exists(CKPT_PATH), f"Không thấy file ckpt: {CKPT_PATH}"

ckpt = torch.load(CKPT_PATH, map_location=device)

# ckpt của anh thường là dict có model_state_dict...
if isinstance(ckpt, dict):
    print("✅ ckpt keys:", list(ckpt.keys())[:20])
    print("epoch:", ckpt.get("epoch"), "val_loss:", ckpt.get("val_loss"))
else:
    print("✅ ckpt is a raw state_dict (not a dict wrapper)")


✅ ckpt keys: ['epoch', 'model_state_dict', 'optimizer_state_dict', 'val_loss']
epoch: 23 val_loss: 2.261958536589542


In [44]:
# ==========================================
# LOAD WEIGHTS INTO EXISTING MODEL
# ==========================================
# 1) Nếu checkpoint là wrapper dict
state_dict = ckpt["model_state_dict"] if isinstance(ckpt, dict) and "model_state_dict" in ckpt else ckpt

# 2) model phải tồn tại từ trước (ưu tiên tái sử dụng)
assert "model" in globals(), "Chưa thấy biến model. Hãy chạy cell tạo model trước (giống lúc train)."

missing, unexpected = model.load_state_dict(state_dict, strict=False)
print("✅ Loaded state_dict")
print("missing keys:", len(missing))
print("unexpected keys:", len(unexpected))

model.to(device)
model.eval()


✅ Loaded state_dict
missing keys: 0
unexpected keys: 0


Transformer(
  (src_emb): Embedding(10000, 256)
  (tgt_emb): Embedding(10000, 256)
  (rope): RotaryPositionalEncoding()
  (encoders): ModuleList(
    (0-3): 4 x TransformerBlock(
      (norm1): RMSNorm((256,), eps=None, elementwise_affine=True)
      (attn): GQA(
        (q_proj): Linear(in_features=256, out_features=256, bias=True)
        (k_proj): Linear(in_features=256, out_features=128, bias=True)
        (v_proj): Linear(in_features=256, out_features=128, bias=True)
        (o_proj): Linear(in_features=256, out_features=256, bias=True)
      )
      (norm_ffn): RMSNorm((256,), eps=None, elementwise_affine=True)
      (ffn): SwiGLU(
        (w1): Linear(in_features=256, out_features=1024, bias=True)
        (w2): Linear(in_features=256, out_features=1024, bias=True)
        (w3): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (decoders): ModuleList(
    (0-3): 4 x TransformerBlock(
      (norm1): RMSNorm((256,), eps=None, elementwise_affine=True)
      (

In [20]:
# ==========================================
# LOAD TEST DATA (EN->VI)
# ==========================================
TEST_SRC_PATH = "/kaggle/input/databaitoanphu/public_test.en.txt"
TEST_TGT_PATH = "/kaggle/input/databaitoanphu/public_test.vi.txt"

assert os.path.exists(TEST_SRC_PATH), f"Không thấy test src: {TEST_SRC_PATH}"
assert os.path.exists(TEST_TGT_PATH), f"Không thấy test tgt: {TEST_TGT_PATH}"

# Nếu notebook đã có hàm read_parallel_data(...) thì dùng luôn
if "read_parallel_data" in globals():
    test_pairs = read_parallel_data(TEST_SRC_PATH, TEST_TGT_PATH)
else:
    # fallback đơn giản (không đụng vào logic chính)
    test_pairs = []
    with open(TEST_SRC_PATH, "r", encoding="utf-8") as fs, open(TEST_TGT_PATH, "r", encoding="utf-8") as ft:
        for s, t in zip(fs, ft):
            s, t = s.strip(), t.strip()
            if s and t:
                test_pairs.append((s, t))

print("✅ #test pairs:", len(test_pairs))
print("sample:", test_pairs[0])


✅ #test pairs: 3000
sample: ('Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao', 'Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017')


In [45]:
# =====================================
# --- FULL: BEAM SEARCH DECODING ---
# =====================================
import torch
import torch.nn.functional as F

@torch.no_grad()
def beam_search_decode(
    model,
    src,                      # (1, src_len)
    beam_size=5,
    max_len=80,
    length_penalty=0.7
):
    """
    Beam search decode for 1 sentence.

    Assumes these globals already exist (as in your notebook):
      - device
      - PAD_ID   (src pad id)
      - START_ID (tgt start id)
      - END_ID   (tgt end id)

    Returns:
      best_tokens: List[int] (includes START_ID, may include END_ID)
    """
    model.eval()
    src = src.to(device)

    # -------------------------
    # 1) Encoder (run once)
    # -------------------------
    src_mask = (src == PAD_ID).unsqueeze(1).unsqueeze(2).float() * -1e9  # (1,1,1,src_len)

    enc = model.src_emb(src)  # (1, src_len, d)
    rope_cos, rope_sin = model.rope(enc, enc.shape[1])

    for layer in model.encoders:
        enc = layer(enc, mask=src_mask, rope_cos=rope_cos, rope_sin=rope_sin)

    # -------------------------
    # 2) Beam init
    # -------------------------
    beams = [([START_ID], 0.0)]   # list of (token_ids, log_prob_sum)
    finished = []                # finished beams that ended with END_ID

    # helper for normalized score
    def norm_score(tokens, score):
        L = max(1, len(tokens))
        return score / (L ** length_penalty)

    # -------------------------
    # 3) Decode steps
    # -------------------------
    for step in range(max_len):
        new_beams = []

        for tokens, score in beams:
            # If ended, keep it
            if tokens[-1] == END_ID:
                finished.append((tokens, score))
                new_beams.append((tokens, score))
                continue

            ys = torch.tensor(tokens, dtype=torch.long, device=device).unsqueeze(0)  # (1, t)

            # causal mask (t, t): -inf for future positions
            tgt_mask = torch.triu(
                torch.full((ys.size(1), ys.size(1)), float("-inf"), device=device),
                diagonal=1
            )

            # ---- Decoder forward ----
            dec = model.tgt_emb(ys)  # (1, t, d)
            rope_cos_t, rope_sin_t = model.rope(dec, dec.shape[1])

            x = dec
            for layer in model.decoders:
                x = layer(
                    x,
                    enc_out=enc,
                    mask=tgt_mask,
                    cross_mask=src_mask,
                    rope_cos=rope_cos_t,
                    rope_sin=rope_sin_t
                )

            logits = model.fc_out(model.final_norm(x))[:, -1, :]  # (1, vocab)
            log_probs = F.log_softmax(logits, dim=-1)             # (1, vocab)

            topk_log_probs, topk_ids = log_probs.topk(beam_size, dim=-1)

            for k in range(beam_size):
                next_id = topk_ids[0, k].item()
                next_score = score + topk_log_probs[0, k].item()
                new_beams.append((tokens + [next_id], next_score))

        # -------------------------
        # 4) Prune (keep best beams)
        # -------------------------
        new_beams = sorted(
            new_beams,
            key=lambda x: norm_score(x[0], x[1]),
            reverse=True
        )

        beams = new_beams[:beam_size]

        # Early stop: if all current beams are finished
        if all(toks[-1] == END_ID for toks, _ in beams):
            break

    # -------------------------
    # 5) Pick best candidate
    # -------------------------
    candidates = finished if finished else beams
    best_tokens, best_score = max(candidates, key=lambda x: norm_score(x[0], x[1]))
    return best_tokens


In [25]:
# ==========================================
# QUICK SANITY CHECK: TRANSLATE FEW SAMPLES
# ==========================================
NUM_SHOW = 5

for i in range(NUM_SHOW):
    src, ref = test_pairs[i]

    # (A) Nếu anh đã có hàm translate_en2vi(text) / translate_sentence(...)
    if "translate_en2vi" in globals():
        pred = translate_en2vi(src)
    elif "translate_sentence" in globals():
        # tuỳ signature của anh: translate_sentence(model, sentence, ...)
        pred = translate_sentence(model, src)
    else:
        # (B) Nếu anh có beam_search_decode(model, src_ids, ...)
        assert "beam_search_decode" in globals() or "greedy_decode" in globals(), \
            "Không thấy hàm translate/beam/greedy có sẵn. Anh chạy cell định nghĩa decode trước."

        # Tokenize theo đúng tokenizer anh đã dùng
        # đổi tên tokenizer cho đúng notebook anh:
        src_ids = en_tokenizer.encode(src).ids
        src_ids = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)

        if "beam_search_decode" in globals():
            out_ids = beam_search_decode(model, src_ids)  # nếu hàm anh signature khác thì chỉnh 1 dòng này
        else:
            out_ids = greedy_decode(model, src_ids)

        # decode target
        pred = vi_tokenizer.decode(out_ids)

    print("="*80)
    print("SRC:", src)
    print("REF:", ref)
    print("PRED:", pred)


SRC: Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao
REF: Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017
PRED:  Kiến thức, thực hành sử dụng dịch vụ y tế công cộng của người dân bảo hiểm y tế và các yếu tố ảnh hưởng tại Vii, Lào Cai
SRC: Describe knowledge, practices in public health service utilization among health insurance card's holders and influencing factors in Vientiane, Lao PDR, 2017.
REF: Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố liên quan tại tỉnh Viêng Chăn, Cộng hoà Dân chủ Nhân dân Lào năm 2017.
PRED:  Mô tả kiến thức, thực hành về sử dụng dịch vụ y tế công cộng của người tham gia bảo hiểm y tế (BHYT) và các yếu tố ảnh hưởng tại Viventan, L

In [47]:
sentence_en = "Group 1: non-obese mice."
sentence_vi = "Nhóm 1: Chuột không béo phì."

print("SRC:", sentence_en)
print("TRG:", sentence_vi)

# 1) Encode EN -> ids tensor
src_ids = en_tokenizer.encode(sentence_en).ids
src = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)  # (1, src_len)

# 2) Beam search -> ids
out_ids = beam_search_decode(model, src, beam_size=5, max_len=80)

# 3) Bỏ START/END trước khi decode text
if len(out_ids) > 0 and out_ids[0] == START_ID:
    out_ids = out_ids[1:]
if END_ID in out_ids:
    out_ids = out_ids[:out_ids.index(END_ID)]

pred_vi = vi_tokenizer.decode(out_ids)
print("PRED:", pred_vi)


SRC: Group 1: non-obese mice.
TRG: Nhóm 1: Chuột không béo phì.
PRED:  Nhóm 1: chuột không thuần chủng.


In [27]:
!pip install torchtext

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall

In [36]:
!pip -q install sacrebleu tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [37]:
# ==========================================
# --- BLEU (SACREBLEU) + PROGRESS BAR ---
# ==========================================


import sacrebleu
from tqdm.auto import tqdm
import torch

MAX_EVAL = len(test_pairs)   # hoặc 2000 để test nhanh

pred_texts = []
ref_texts  = []

for i in tqdm(range(MAX_EVAL), desc="Computing BLEU", unit="sent"):
    src, ref = test_pairs[i]

    # dùng hàm dịch có sẵn của anh (ưu tiên)
    if "translate_en2vi" in globals():
        pred = translate_en2vi(src)
    elif "translate_sentence" in globals():
        pred = translate_sentence(model, src)
    else:
        # fallback nếu anh chỉ có beam_search_decode/greedy_decode
        src_ids = en_tokenizer.encode(src).ids
        src_ids = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)
        out_ids = beam_search_decode(model, src_ids) if "beam_search_decode" in globals() else greedy_decode(model, src_ids)

        # bỏ START/END nếu cần (tuỳ model anh)
        if isinstance(out_ids, torch.Tensor):
            out_ids = out_ids.squeeze(0).tolist() if out_ids.ndim > 1 else out_ids.tolist()

        if len(out_ids) > 0 and out_ids[0] == START_ID:
            out_ids = out_ids[1:]
        if END_ID in out_ids:
            out_ids = out_ids[:out_ids.index(END_ID)]

        pred = vi_tokenizer.decode(out_ids)

    pred_texts.append(pred.strip())
    ref_texts.append(ref.strip())

bleu = sacrebleu.corpus_bleu(pred_texts, [ref_texts])
print("BLEU =", bleu.score)
print("detail:", bleu)


Computing BLEU:   0%|          | 0/3000 [00:00<?, ?sent/s]

BLEU = 46.57571741460038
detail: BLEU = 46.58 75.8/58.4/46.4/37.7 (BP = 0.883 ratio = 0.890 hyp_len = 89734 ref_len = 100870)


Tính Bleu khi train ở epoch: 23 val_loss: 2.261958536589542 

In [48]:
# ==========================================
# --- BLEU (SACREBLEU) + PROGRESS BAR ---
# ==========================================


import sacrebleu
from tqdm.auto import tqdm
import torch

MAX_EVAL = len(test_pairs)   # hoặc 2000 để test nhanh

pred_texts = []
ref_texts  = []

for i in tqdm(range(MAX_EVAL), desc="Computing BLEU", unit="sent"):
    src, ref = test_pairs[i]

    # dùng hàm dịch có sẵn của anh (ưu tiên)
    if "translate_en2vi" in globals():
        pred = translate_en2vi(src)
    elif "translate_sentence" in globals():
        pred = translate_sentence(model, src)
    else:
        # fallback nếu anh chỉ có beam_search_decode/greedy_decode
        src_ids = en_tokenizer.encode(src).ids
        src_ids = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)
        out_ids = beam_search_decode(model, src_ids) if "beam_search_decode" in globals() else greedy_decode(model, src_ids)

        # bỏ START/END nếu cần (tuỳ model anh)
        if isinstance(out_ids, torch.Tensor):
            out_ids = out_ids.squeeze(0).tolist() if out_ids.ndim > 1 else out_ids.tolist()

        if len(out_ids) > 0 and out_ids[0] == START_ID:
            out_ids = out_ids[1:]
        if END_ID in out_ids:
            out_ids = out_ids[:out_ids.index(END_ID)]

        pred = vi_tokenizer.decode(out_ids)

    pred_texts.append(pred.strip())
    ref_texts.append(ref.strip())

bleu = sacrebleu.corpus_bleu(pred_texts, [ref_texts])
print("BLEU =", bleu.score)
print("detail:", bleu)


Computing BLEU:   0%|          | 0/3000 [00:00<?, ?sent/s]

BLEU = 44.982471593436834
detail: BLEU = 44.98 74.8/56.8/44.3/35.4 (BP = 0.886 ratio = 0.892 hyp_len = 89958 ref_len = 100870)
